# Purpose:
- Analyze hydration-driven z-drift divergence in cre lines other than GAD2-Cre
- From VisualBehaviorMultiscope, 2x4 configuration


In [1]:
import brain_observatory_qc.pipeline_dev.scripts.depth_estimation_module as dem
from pathlib import Path
import numpy as np
import pandas as pd
import h5py
from datetime import datetime
from matplotlib import pyplot as plt
from allensdk.brain_observatory.behavior.behavior_project_cache import VisualBehaviorOphysProjectCache as bpc
from brain_observatory_qc.data_access import from_lims
from pymongo import MongoClient
from visual_behavior import database as db
from datetime import datetime


mongo = MongoClient('flaskapp.corp.alleninstitute.org', 27017)

save_dir = Path(r'\\allen\programs\mindscope\workgroups\learning\ophys\zdrift')


def get_depth_region_from_db(oeid):
    query = '''
    SELECT oe.id, st.acronym, oe.calculated_depth, imd.depth
    FROM ophys_experiments oe
    JOIN structures st ON st.id = oe.targeted_structure_id
    JOIN imaging_depths imd ON imd.id = oe.imaging_depth_id
    WHERE oe.id = {}
    '''.format(oeid)
    lims_results = db.lims_query(query)
    return lims_results.depth.values[0], lims_results.acronym.values[0]


def get_zdrift_results(osid, load_dir):
    oeids = from_lims.get_ophys_experiment_ids_for_ophys_session_id(osid).ophys_experiment_id.values
    corrcoef = []
    matched_plane_indices = []
    for oeid in oeids:
        try:
            ocid = from_lims.get_ophys_container_id_for_ophys_experiment_id(oeid)
        except:
            ocid = 'none'
        zdrift_fn = load_dir / f'container_{ocid}' / f'experiment_{oeid}' / f'{oeid}_zdrift_ref_{oeid}.h5'
        
        with h5py.File(zdrift_fn) as h:
            corrcoef.append([np.max(cc) for cc in h['corrcoef'][:]])
            matched_plane_indices.append(h['matched_plane_indices'][:])
    return corrcoef, matched_plane_indices, oeids

def get_zdrift_results_first_last(osid, load_dir):
    oeids = from_lims.get_ophys_experiment_ids_for_ophys_session_id(osid).ophys_experiment_id.values
    corrcoef = []
    matched_plane_indices = []
    for oeid in oeids:
        try:
            ocid = from_lims.get_ophys_container_id_for_ophys_experiment_id(oeid)
        except:
            ocid = 'none'
        zdrift_fn = load_dir / f'container_{ocid}' / f'experiment_{oeid}' / f'{oeid}_zdrift_ref_{oeid}_first_last.h5'
        with h5py.File(zdrift_fn) as h:
            corrcoef.append([np.max(cc) for cc in h['corrcoef'][:]])
            matched_plane_indices.append(h['matched_plane_indices'][:])
    return corrcoef, matched_plane_indices, oeids


# Select dataset

In [45]:
cache = bpc.from_lims()
exp_table = cache.get_ophys_experiment_table(passed_only=False)
vbms_table = exp_table.query('project_code == "VisualBehaviorMultiscope"')
session_ids = vbms_table.groupby('ophys_session_id').size()
session_ids = session_ids[session_ids == 8].index
table = vbms_table.query('ophys_session_id in @session_ids')
print(len(table))

2080


In [12]:
table.groupby('session_type').size()

session_type
OPHYS_0_images_A_habituation         8
OPHYS_1_images_A                   328
OPHYS_2_images_A_passive           304
OPHYS_2_images_B_passive             8
OPHYS_3_images_A                   336
OPHYS_4_images_B                   352
OPHYS_5_images_A_passive             8
OPHYS_5_images_B_passive           232
OPHYS_6_images_A                     8
OPHYS_6_images_B                   272
OPHYS_7_receptive_field_mapping    216
SNR_9                                8
dtype: int64

In [13]:
table = table[table.session_type.str.contains('OPHYS_') & ~table.session_type.str.contains('OPHYS_0')]

In [14]:
table.groupby('full_genotype').apply(lambda x: len(x.mouse_id.unique()))

full_genotype
Slc17a7-IRES2-Cre/wt;Camk2a-tTA/wt;Ai93(TITL-GCaMP6f)/wt    11
Sst-IRES-Cre/wt;Ai148(TIT2L-GC6f-ICL-tTA2)/wt                9
Vip-IRES-Cre/wt;Ai148(TIT2L-GC6f-ICL-tTA2)/wt               11
dtype: int64

In [42]:
# prepare dataframe to be passed to multi_plane_drift_sbatch.py
table = table[['mouse_id','ophys_session_id', 'full_genotype', 'session_type']].drop_duplicates()
for osid in table.ophys_session_id.values:
    session = mongo.qc.metrics.find_one({'lims_id': int(osid)})
    if 'change_detection' not in session:
        if table.query('ophys_session_id == @osid').session_type.str.contains('OPHYS_7').values[0]:
            water = 0
        elif table.query('ophys_session_id == @osid').session_type.str.contains('passive').values[0]:
            water = 0
        else:
            raise ValueError(f'no change detection data for {osid}')
    else:
        water = session['change_detection']['total_reward_volume']
    table.loc[table.ophys_session_id == osid, 'water'] = water
    

In [44]:
table.to_csv(save_dir / 'multi_plane_zdrift_vb_multiscope.csv', index=False)

In [5]:
load_csv = save_dir / 'multi_plane_zdrift_vb_multiscope.csv'
table = pd.read_csv(load_csv)
len(table)

258

# Some sessions errored out
- 23 / 258 sessions errored out
- Just use the remaining 235 sessions

In [3]:
job_records_dir = Path(r'\\allen\programs\mindscope\workgroups\learning\ophys\zdrift\job_records')

# filter by created time, if needed
time_since = datetime(2024, 3, 26, 15, 0, 0)
# from job_dir get files created after time_since
files = [f for f in job_records_dir.glob('**/*') if f.is_file() and f.stat().st_ctime > time_since.timestamp()]

failed_files = []
failed_osids = []
success_word = 'total time ='
for file in files:
    with open(file, 'r') as f:
        lines = f.readlines()
        if np.array([success_word in l for l in lines]).any():
            pass
        else:
            failed_files.append(file)
            failed_osids.append(int(file.name.split('_')[-1].split('.')[0]))
failed_osids = np.unique(failed_osids)

In [4]:
failed_osids

array([ 841778484,  842023261,  842364341,  843871999,  844469521,
        852070825,  856201876,  870762788,  871906231,  873247524,
        873720614,  874616920,  875259383,  917886781,  918180797,
        924644503,  927620400,  942628468,  974167263,  976061108,
       1001385706, 1047935100, 1050231786])

In [6]:
load_csv = save_dir / 'multi_plane_zdrift_vb_multiscope.csv'
table = pd.read_csv(load_csv)
table = table[~table.ophys_session_id.isin(failed_osids)]
len(table)

235

In [7]:
table.head()

,mouse_id,ophys_session_id,full_genotype,session_type,water
0,457841,952430817,Sst-IRES-Cre/wt;Ai148(TIT2L-GC6f-ICL-tTA2)/wt,OPHYS_2_images_A_passive,0.000
1,457841,954954402,Sst-IRES-Cre/wt;Ai148(TIT2L-GC6f-ICL-tTA2)/wt,OPHYS_3_images_A,0.396
2,457841,960475921,Sst-IRES-Cre/wt;Ai148(TIT2L-GC6f-ICL-tTA2)/wt,OPHYS_7_receptive_field_mapping,0.000
3,457841,957020350,Sst-IRES-Cre/wt;Ai148(TIT2L-GC6f-ICL-tTA2)/wt,OPHYS_4_images_B,0.515
4,457841,955775716,Sst-IRES-Cre/wt;Ai148(TIT2L-GC6f-ICL-tTA2)/wt,OPHYS_3_images_A,0.445


## example time-series plot
- One example per cre line, divided into VISp and VISl, with or without water